In [2]:
# Гипотеза и вывод
import math
import random


# code from probability_theory.ipynb

def normal_cdf(x, mu=0, sigma=1):
    """
    ИРФ Нормального распределения
    """
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma )) / 2


def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """
    Найти приближенную инверсию, используя двоичный поиск
    """
    # если не стандартизированно, то стандартизировать и прошкалировать
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z, low_p = -10, 0
    hi_z, hi_p = 10, 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2
        mid_p = (low_z + hi_z) / 2
        if mid_p < p:
            #значение середины все еще слишком низкое, искать выше
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            #значение середины все еще слишком высокое - искать ниже
            hi_z, hi_p = mid_z, mid_p
        else:
            break
    
    return mid_z


# new code

def normal_approximation_to_binominal(n, p):
    """
    Аппроксимация биноминальной случайной величины нормальным распределением
    """
    # находим mu и sigma, которые соответствуют binominal(n, p)
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    
    return mu, sigma


# вероятность, что значение нормальной случайной величины лежит ниже порогового значения
normal_probability_below = normal_cdf


def normal_probability_above(lo, mu=0, sigma=1):
    """
    оно выше порогового значения, если оно не ниже
    """
    return 1 - normal_cdf(lo, mu, sigma)


def normal_probability_between(lo, hi, mu=0, sigma=1):
    """
    оно лежитм между, если меньше hi и больше lo
    """
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)


def normal_probabiliy_outside(lo, hi, mu, sigma=1):
    """
    оно лежит за пределами, если оно не внутри
    """
    return 1 - normal_probability_between(lo, hi, mu, sigma)


def normal_upper_bound(probability, mu=0, sigma=1):
    """
    верхняя граница нормальной величины
    возвращает z, для которого P(Z <= z) = probability
    """
    return inverse_normal_cdf(probability, mu, sigma)


def normal_lower_bound(probability, mu=0, sigma=1):
    """
    Нижняя граница нормальной величины
    Возвращает z, для которого P(Z >= z) = probability
    """
    return inverse_normal_cdf(1-probability, mu, sigma)
    

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """
    Возвращает симметричные (вокруг среднего значения) границы,
    в пределах которых содержится указанная вероятность
    """
    tail_probability = (1 - probability) / 2
    
    # верхняя граница должна иметь значение хвостовой вероятности
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # нижняя граница должна иметь значение хвостовой вероятности
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound


# 1000 бросков монеты -> (среднее значение, стандартное отклонение)
mu_0, sigma_0 = normal_approximation_to_binominal(1000, 0.5)
print(mu_0, sigma_0)

500.0 15.811388300841896


In [3]:
# 95% границы, при условии, что p=0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(lo, hi)

500.39521836029536 515.4162302562063


In [4]:
# фактические mu и sigma при p=0.55
mu_1, sigma_1 = normal_approximation_to_binominal(1000, 0.55)
print(mu_1, sigma_1)

550.0 15.732132722552274


In [5]:
# Ошибка второго рода означает: не удалось отклонить нулевую гипотезу
# это происходит, когда X все еще внутри первоначального интервала
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(f'Мощность = {power}')

Мощность = 0.986843499293525


In [6]:
# верхняя граница нормальной случайной величины
hi = normal_upper_bound(0.95, mu_0, sigma_0)
print(hi)

515.0208611067616


In [7]:
# вероятность ошибки второго рода
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(f'Мощность = {power}')

Мощность = 0.9869062545985033


In [8]:
# P-значения

def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        #если x больше среднего значения, то значения в хвосте больше x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        #если x меньше среднего значения, то значения в хвосте меньше x
        return 2 * normal_probability_below(x, mu, sigma)
    

print(two_sided_p_value(529.5, mu_0, sigma_0))

0.06207721579598835


In [ ]:
# чтобы убедиться, что результат разумен, проводим модельное испытание
extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0 for _ in range(100000))
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1

print(extreme_value_count)

In [ ]:
# Для проведения односторонней проверки при выпадении 525 орлов получаем
print(upper_p_value(524.5, mu_0, sigma_0))

In [ ]:
# при выпадении 547 орлов получим
print(upper_p_value(526.5, mu_0, sigma_0))

In [ ]:
# Доверительные интервалы
# среднее случайных величин с распределением Бернулли
# average_limit = math.sqrt(p * (1 -p) / 1000)

# по скольку значение p неизвестно используется другая оценка
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print(normal_two_sided_bounds(0.95, mu, sigma))